Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import tensorflow.keras as keras
from keras import layers
from keras import callbacks
from loguru import logger
from sklearn.model_selection import train_test_split

In [2]:
TEST_SAMPLE_FILE_PATH = 'data/fashion-mnist_test.csv'
TRAIN_SAMPLE_FILE_PATH = 'data/fashion-mnist_train.csv'

test_sample_df = pd.read_csv(TEST_SAMPLE_FILE_PATH)
train_sample_df = pd.read_csv(TRAIN_SAMPLE_FILE_PATH)

Подготовка данных

In [3]:
train_data = np.array(train_sample_df, dtype='float32')
test_data = np.array(test_sample_df, dtype='float32')

x_train = train_data[:, 1:] / 255
y_train = train_data[:, 0]

x_test = test_data[:, 1:] / 255
y_test = test_data[:, 0]

x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.2, random_state=228)

Настройки модели

In [4]:

img_rows = 28
img_cols = 28

filters = 32
kernel_size = 3
activation = 'relu'
pool_size = 2
dropout = 0.2
dense = 32
dense_activation = 'relu'
output = 10
output_activation = 'softmax'

number_epochs = 17
verbose = 1
batch_size = 512
loss = 'sparse_categorical_crossentropy'
metrics = ['accuracy']
    
write_graph = True
histogram_freq = 1
write_images = True


x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_validate = x_validate.reshape(x_validate.shape[0], img_rows, img_cols, 1)

logger.info(f"Размер обучающей выборки: {x_train.shape}")
logger.info(f"Размер тестовой выборки: {x_test.shape}")
logger.info(f"Размер валидационной выборки: {x_validate.shape}")


model_layers = (
    layers.Input(shape=(img_rows, img_cols, 1)),
    layers.Conv2D(
        filters=filters,
        kernel_size=kernel_size, 
        activation=activation,
    ),
    layers.MaxPooling2D(pool_size=pool_size),
    layers.Dropout(dropout),
    layers.Flatten(),
    layers.Dense(dense, activation=dense_activation),
    layers.Dense(output, activation=output_activation)
)

model = keras.Sequential(model_layers)

tensorboard = callbacks.TensorBoard(
    write_graph=write_graph,
    histogram_freq=histogram_freq,
    write_images=write_images,
)

model.compile(
    loss=loss,
    metrics=metrics,
)

model.fit(
    x_train, 
    y_train, 
    batch_size=batch_size,
    epochs=number_epochs, 
    verbose=verbose,
    validation_data=(x_validate, y_validate),
    callbacks=[tensorboard]
)

_, accuracy, *_ = model.evaluate(x_test, y_test, verbose=verbose)

if accuracy > 0.8:
    logger.success(f"Точность модели: {accuracy}")
    
    y_response = model(x_test)
    
    result = pd.DataFrame(np.argmax(y_response, axis=1), columns=["Category"])
    result["Id"] = result.index

    result.to_csv("data/result_submission.csv", index=False)
else:
    logger.warning(f"Точность модели: {accuracy} < 0.8")

2024-05-13 01:14:19.656 | INFO     | __main__:<module>:29 - Размер обучающей выборки: (48000, 28, 28, 1)
2024-05-13 01:14:19.657 | INFO     | __main__:<module>:30 - Размер тестовой выборки: (10000, 28, 28, 1)
2024-05-13 01:14:19.658 | INFO     | __main__:<module>:31 - Размер валидационной выборки: (12000, 28, 28, 1)


Epoch 1/17
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.5782 - loss: 1.2455 - val_accuracy: 0.8054 - val_loss: 0.5635
Epoch 2/17
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.7997 - loss: 0.5635 - val_accuracy: 0.8019 - val_loss: 0.5182
Epoch 3/17
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.8296 - loss: 0.4696 - val_accuracy: 0.8461 - val_loss: 0.4319
Epoch 4/17
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.8510 - loss: 0.4163 - val_accuracy: 0.8465 - val_loss: 0.4148
Epoch 5/17
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.8668 - loss: 0.3793 - val_accuracy: 0.8630 - val_loss: 0.3806
Epoch 6/17
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.8716 - loss: 0.3586 - val_accuracy: 0.8807 - val_loss: 0.3419
Epoch 7/17
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.8810 - loss: 0.3371 - val_accuracy: 0.8671 - val_loss: 0.3636
Epoch 8/17
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.8860 - loss: 0.3192 - val_accuracy: 0.8824 - v

2024-05-13 01:15:06.868 | SUCCESS  | __main__:<module>:74 - Точность модели: 0.9100000262260437
